# Lesson 4.5: Practical Application - Building an API-Interacting Agent

---

In previous lessons, we learned the theory behind **Agents** and how they use **Tools** to extend the capabilities of Large Language Models (LLMs). Now it's time to put that knowledge into practice by building an Agent capable of interacting with an external API. This practical exercise will guide you step-by-step through creating a **Custom Tool** and integrating it into an Agent to solve real-world tasks.

## 1. Objectives of this Practical Exercise

The main objectives of this practical exercise are to:
* Reinforce knowledge about how to create and use **Custom Tools**.
* Build an **Agent** capable of making decisions and using custom Tools to interact with a public API.
* Gain a deeper understanding of the Agent's reasoning and action flow when working with Tools.




---

## 2. Environment Setup and API Selection

To begin, ensure you have installed the necessary libraries and set up your API key for your LLM model.

**Required Libraries:**
```bash
pip install langchain-openai openai requests pydantic
```
* `langchain-openai`: LangChain integration with OpenAI.
* `openai`: Official OpenAI Python library.
* `requests`: For making HTTP requests to external APIs.
* `pydantic`: For defining input schemas for Tools (optional, but recommended).

**API Key Setup:**
```python
import os
# Set your API key in an environment variable
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
```

**Public API Selection:**
We will use the **Quotable API** (`https://api.quotable.io/random`) to fetch random quotes. This is a simple API that does not require authentication or API keys, making it ideal for illustration purposes.
To add versatility, we will also reuse the mock weather lookup tool from the previous lesson.


---

## 3. Writing Custom Tools to Interact with APIs

We will create two Custom Tools: one to fetch a random quote and one for mock weather lookup.

### 3.1. Custom Tool: Get Random Quote (`get_random_quote`)

This Tool will call the Quotable API and return a random quote along with its author.

In [ ]:
import requests
from langchain.tools import tool

@tool
def get_random_quote() -> str:
    """
    Lấy một câu danh ngôn ngẫu nhiên từ Quotable API.
    Không cần đầu vào.
    Trả về một chuỗi chứa câu danh ngôn và tác giả.
    """
    try:
        response = requests.get("https://api.quotable.io/random")
        response.raise_for_status() # Ném lỗi cho các mã trạng thái HTTP xấu (4xx hoặc 5xx)
        data = response.json()
        content = data.get("content", "Không có nội dung câu danh ngôn.")
        author = data.get("author", "Không rõ tác giả.")
        return f"Câu danh ngôn: '{content}' - Tác giả: {author}"
    except requests.exceptions.RequestException as e:
        return f"Lỗi khi lấy câu danh ngôn: {e}"

# Kiểm tra Tool tùy chỉnh
print("--- Kiểm tra Custom Quote Tool ---")
print(get_random_quote())
print("-" * 30)

### 3.2. Custom Tool: Mock Weather Lookup (`get_weather_data`)

This is a Tool similar to the one in Lesson 4.4, used to illustrate that an Agent can use multiple types of Tools.

In [ ]:
@tool
def get_weather_data(location: str) -> str:
    """
    Lấy thông tin thời tiết hiện tại cho một địa điểm cụ thể.
    Đầu vào là tên thành phố hoặc địa điểm (ví dụ: "Hà Nội", "Đà Nẵng", "Tokyo").
    Trả về chuỗi mô tả thời tiết.
    """
    location_lower = location.lower()
    if "hanoi" in location_lower:
        return "Thời tiết Hà Nội: 28°C, trời nắng, độ ẩm 70%, gió nhẹ."
    elif "da nang" in location_lower:
        return "Thời tiết Đà Nẵng: 30°C, trời quang đãng, độ ẩm 65%, không có mưa."
    elif "london" in location_lower:
        return "Thời tiết London: 15°C, nhiều mây, có mưa phùn, gió mạnh."
    elif "tokyo" in location_lower:
        return "Thời tiết Tokyo: 25°C, nắng đẹp, độ ẩm 60%."
    else:
        return f"Không tìm thấy thông tin thời tiết cho địa điểm: {location}."

# Kiểm tra Tool tùy chỉnh
print("--- Kiểm tra Custom Weather Tool ---")
print(get_weather_data("Hanoi"))
print(get_weather_data("Sydney"))
print("-" * 30)


---

## 4. Building an Agent Using Custom Tools

Now, we will combine the LLM with these Custom Tools to create an Agent capable of making decisions and using them.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.tools import Tool

# Thiết lập biến môi trường cho khóa API của OpenAI
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# 1. Khởi tạo LLM
# Sử dụng temperature thấp để Agent ra quyết định ổn định hơn
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 2. Tập hợp danh sách các Tools mà Agent có thể sử dụng
tools = [
    get_random_quote, # Tool lấy câu danh ngôn
    get_weather_data  # Tool tra cứu thời tiết
]

# 3. Định nghĩa Prompt cho Agent
# Prompt này sẽ hướng dẫn LLM cách suy nghĩ và sử dụng các công cụ.
# MessagesPlaceholder(variable_name="agent_scratchpad") là rất quan trọng
# để Agent ghi lại các Thought, Action, Observation của mình.
prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là một trợ lý hữu ích. Bạn có quyền truy cập vào các công cụ sau: {tools}. Sử dụng chúng để trả lời các câu hỏi. Nếu câu hỏi không liên quan đến thời tiết hoặc danh ngôn, hãy trả lời bằng kiến thức chung của bạn."),
    MessagesPlaceholder(variable_name="chat_history"), # Để duy trì lịch sử trò chuyện
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 4. Tạo Agent
# create_react_agent tạo một Zero-shot ReAct Agent
agent = create_react_agent(llm, tools, prompt)

# 5. Tạo Agent Executor
# Cấu hình max_iterations để ngăn vòng lặp vô hạn
# Cấu hình handle_parsing_errors để Agent có thể tự sửa lỗi
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True, # Để xem quá trình suy nghĩ của Agent
    max_iterations=5, # Giới hạn số bước để tránh vòng lặp vô hạn
    handle_parsing_errors=True # Cho phép Agent cố gắng tự sửa lỗi phân tích cú pháp
)

print("\nĐã xây dựng Agent với Custom Tools.")


---

## 5. Executing and Testing the Agent

Now, let's try asking a few questions to your Agent to see how it interacts with the APIs.

In [ ]:
# --- Thực thi Agent ---
print("\n--- Bắt đầu kiểm tra Agent tương tác API ---") # Starting API-interacting Agent test
chat_history = []

# Câu hỏi 1: Yêu cầu lấy câu danh ngôn ngẫu nhiên
query_1 = "Hãy cho tôi một câu danh ngôn ngẫu nhiên."
print(f"\nNgười dùng: {query_1}") # User:
response_1 = agent_executor.invoke({"input": query_1, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=query_1), AIMessage(content=response_1["output"])])
print(f"Agent: {response_1['output']}")

# Câu hỏi 2: Yêu cầu tra cứu thời tiết
query_2 = "Thời tiết ở Tokyo hôm nay thế nào?"
print(f"\nNgười dùng: {query_2}") # User:
response_2 = agent_executor.invoke({"input": query_2, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=query_2), AIMessage(content=response_2["output"])])
print(f"Agent: {response_2['output']}")

# Câu hỏi 3: Câu hỏi không liên quan đến Tools, LLM sẽ trả lời bằng kiến thức chung
query_3 = "Thủ đô của Canada là gì?"
print(f"\nNgười dùng: {query_3}") # User:
response_3 = agent_executor.invoke({"input": query_3, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=query_3), AIMessage(content=response_3["output"])])
print(f"Agent: {response_3['output']}")

# Câu hỏi 4: Yêu cầu lấy câu danh ngôn một lần nữa
query_4 = "Cho tôi một câu danh ngôn khác."
print(f"\nNgười dùng: {query_4}") # User:
response_4 = agent_executor.invoke({"input": query_4, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=query_4), AIMessage(content=response_4["output"])])
print(f"Agent: {response_4['output']}")

print("\n--- Kết thúc kiểm tra Agent tương tác API ---") # Ending API-interacting Agent test


**Explanation:**
* When you ask for a quote, the Agent will recognize that the `get_random_quote` Tool is appropriate and invoke it.
* When you ask about the weather, the Agent will invoke the `get_weather_data` Tool.
* For questions unrelated to the Tools' functionality (e.g., "What is the capital of Canada?"), the Agent will not find a suitable Tool and will attempt to answer using the LLM's internal knowledge.
* Using `verbose=True` in `AgentExecutor` will help you observe the Agent's detailed thought process (`Thought`), actions (`Action`), and observations (`Observation`), giving you a clearer understanding of how it makes decisions.


---

## Lesson Summary

In this lesson, you practiced building an **Agent** capable of interacting with **external APIs** through **Custom Tools**. You learned how to:
* Select a public API and design a **Custom Tool** to call it (e.g., `get_random_quote` for the Quotable API).
* Reuse and integrate other Custom Tools (e.g., `get_weather_data`).
* **Build an Agent** by connecting an LLM with a list of Tools and a carefully designed Prompt.
* **Execute the Agent** and observe how it autonomously selects and uses the appropriate Tools to answer API-related questions.

This practical exercise has reinforced your knowledge of Agents and Tools, allowing you to extend the capabilities of LLM applications to interact with the external world in a flexible and intelligent manner.